<a href="https://colab.research.google.com/github/Junhojuno/tensorflow-tutorial-with-colab/blob/master/2_cnn_mnist_tensorflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
# data loading
train_images = mnist.train.images
train_labels = mnist.train.labels
test_images = mnist.test.images
test_labels = mnist.test.labels

In [4]:
n_input = 784
n_channel = 64 # filter 갯수
n_classes = 10

x = tf.placeholder(dtype='float', shape=[None, n_input])
y = tf.placeholder(dtype='float', shape=[None, n_classes])

# setting parameters
# filter size는 3x3이고 data가 흑백이므로 3x3x1 filter weights 
weights = {
    'convolution1' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,1,n_channel], stddev=0.1)),
    'convolution2' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,n_channel,128], stddev=0.1)),
    'dense' : tf.Variable(initial_value=tf.random_normal(shape=[7*7*128,n_classes], stddev=0.1))
}

biases = {
    'convolution1' : tf.Variable(initial_value=tf.random_normal(shape=[n_channel], stddev=0.1)),
    'convolution2' : tf.Variable(initial_value=tf.random_normal(shape=[128], stddev=0.1)),
    'dense' : tf.Variable(initial_value=tf.random_normal(shape=[n_classes], stddev=0.1))
}

print("parameters setting...!")

parameters setting...!


In [0]:
# network setting
def CNN(x,weights, biases):
  
  # 1st layer
  # input reshape
  x = tf.reshape(x,shape=[-1,28,28,1])
  # convolution! + bias
  conv1 = tf.nn.conv2d(x, filter=weights['convolution1'], strides=[1,1,1,1], padding='SAME')
  conv1 = tf.nn.bias_add(conv1, biases['convolution1'])
  # activation function
  conv1 = tf.nn.relu(conv1)
  # max pooling with non-overlapping
  conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
  
  # 2nd layer
  conv2 = tf.nn.conv2d(conv1, filter=weights['convolution2'], strides=[1,1,1,1], padding='SAME')
  conv2 = tf.nn.bias_add(conv2, bias=biases['convolution2'])
  conv2 = tf.nn.relu(conv2)
  conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
  
  # fully connected layer(dense layer)
  # 한줄로 펴기 -1 : 데이터 갯수
  dense = tf.reshape(conv2, shape=[-1,weights['dense'].get_shape().as_list()[0]])
  logit = tf.add(tf.matmul(dense,weights['dense']), biases['dense'])
  
  return logit


cnn_output = CNN(x, weights, biases)

# cost(loss) & optimizer &accuracy setting
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=cnn_output))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
correct = tf.equal(tf.argmax(cnn_output,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct,'float'))

In [8]:
training_epochs = 20
batch_size = 100
display_step = 4

# Launch the Graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Optimizer
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    # Iteration
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optimizer, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # Display
    if (epoch + 1) % display_step == 0:
        print("Epoch: %03d/%03d cost: %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accuracy, feed_dict=feeds)
        print("Train Accuracy: %.3f" % (train_acc))
        feeds = {x: test_images[:100, :], y: test_labels[:100, :]}
        test_acc = sess.run(accuracy, feed_dict=feeds)
        print("Test Accuracy: %.3f" % (test_acc))
    
print("Optimization Finished!!!")

Epoch: 004/020 cost: 0.027440274
Train Accuracy: 0.980
Test Accuracy: 0.990
Epoch: 008/020 cost: 0.009395549
Train Accuracy: 1.000
Test Accuracy: 0.980
Epoch: 012/020 cost: 0.003462189
Train Accuracy: 1.000
Test Accuracy: 0.980
Epoch: 016/020 cost: 0.001500432
Train Accuracy: 1.000
Test Accuracy: 0.980
Epoch: 020/020 cost: 0.001563977
Train Accuracy: 1.000
Test Accuracy: 0.980
Optimization Finished!!!
